# Rational Speech Act Framework

This notebook implements the basic scalar implicature game as described in [Degen, J. (2023)](https://www.annualreviews.org/docserver/fulltext/linguistics/9/1/annurev-linguistics-031220-010811.pdf?expires=1712101300&id=id&accname=guest&checksum=77D4FE3CFC55DEAC0AA30F6F193CB190).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Matplotlib created a temporary cache directory at /tmp/matplotlib-mxrzw2s0 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def normalize(x):
    return x / np.sum(x, axis=0)

def safe_log(x, eps=1e-10):
    clipped_x = np.clip(x, eps, None)
    return np.where(x > 0, np.log(clipped_x), -np.inf)

In [7]:
class RationalSpeechAgent:
    
    def __init__(self, world_states, utterances, prior=None):
        self.world_states = world_states
        self.utterances = utterances
        self.prior = prior if prior is not None else np.ones(len(world_states)) / len(world_states)
        self.literal_listener_matrix = self.initialize_literal_listener_matrix()

    def initialize_literal_listener_matrix(self):
        matrix = np.array([
            [1.0, 0.0, 0.0],  # m_0
            [0.0, 1.0, 0.0],  # m_1
            [0.0, 1.0, 0.0],  # m_2
            [0.0, 1.0, 0.0],  # m_3
            [0.0, 1.0, 1.0],  # m_4
        ])
        return normalize(matrix)

    def literal_listener(self, utterance_index):
        return self.literal_listener_matrix[:, utterance_index]

    def pragmatic_speaker(self, world_state_index, alpha=1.0, cost=0):
        U = lambda u: safe_log(alpha * self.literal_listener_matrix[u, :]) - cost
        return normalize(np.exp(alpha * U(world_state_index)))

    def pragmatic_listener(self, utterance_index):
        pragmatic_speaker_matrix = np.array([self.pragmatic_speaker(index) for index, world_state in enumerate(self.world_states)])
        return normalize(pragmatic_speaker_matrix[:, utterance_index] * self.prior)

In [8]:
# Instantiate the agent with the given world states and utterances
agent = RationalSpeechAgent(
    world_states=['m_0', 'm_1', 'm_2', 'm_3', 'm_4'],
    utterances=['none', 'some', 'all']
)

# Test the literal listener's output
print("Testing Literal Listener")
for idx, utterance in enumerate(agent.utterances):
    result = agent.literal_listener(idx)
    print(f"Literal listener probabilities for '{utterance}': {result}")

Testing Literal Listener
Literal listener probabilities for 'none': [1. 0. 0. 0. 0.]
Literal listener probabilities for 'some': [0.   0.25 0.25 0.25 0.25]
Literal listener probabilities for 'all': [0. 0. 0. 0. 1.]


In [9]:
print("\nTesting Pragmatic Speaker")
for idx, state in enumerate(agent.world_states):
    result = agent.pragmatic_speaker(idx)
    print(f"Pragmatic speaker probabilities for world state '{state}': {result}")


Testing Pragmatic Speaker
Pragmatic speaker probabilities for world state 'm_0': [1. 0. 0.]
Pragmatic speaker probabilities for world state 'm_1': [0. 1. 0.]
Pragmatic speaker probabilities for world state 'm_2': [0. 1. 0.]
Pragmatic speaker probabilities for world state 'm_3': [0. 1. 0.]
Pragmatic speaker probabilities for world state 'm_4': [0.  0.2 0.8]


In [10]:
print("\nTesting Pragmatic Listener")
for idx, utterance in enumerate(agent.utterances):
    result = agent.pragmatic_listener(idx)
    print(f"Pragmatic listener updated beliefs for utterance '{utterance}': {result}")


Testing Pragmatic Listener
Pragmatic listener updated beliefs for utterance 'none': [1. 0. 0. 0. 0.]
Pragmatic listener updated beliefs for utterance 'some': [0.     0.3125 0.3125 0.3125 0.0625]
Pragmatic listener updated beliefs for utterance 'all': [0. 0. 0. 0. 1.]
